# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [23]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [176]:
grouped = data.groupby(["CustomerID","ProductName"])["Quantity"].sum().reset_index()
grouped

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [70]:
pivot = pd.pivot_table(data,index=['ProductName'],columns=['CustomerID'],values="Quantity",fill_value=0)
pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [88]:
from scipy.spatial.distance import pdist, squareform

distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
                         index=pivot.columns, columns=pivot.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003732,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003638,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003732,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [208]:
CUSTOMERID = 639

similarities = distances[CUSTOMERID].sort_values(ascending=False)[1:]
top_5 = similarities[:5]
top_5

CustomerID
1920    0.094414
3253    0.093498
3317    0.093498
649     0.093498
3472    0.093051
Name: 639, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [209]:
records = grouped[grouped["CustomerID"].isin(top_5.index.to_list())]
records

,CustomerID,ProductName,Quantity
510,649,Arizona - Green Tea,1
511,649,Bacardi Breezer - Tropical,1
512,649,Beans - Kidney White,1
513,649,"Beans - Kidney, Red Dry",1
514,649,"Beef - Chuck, Boneless",1
...,...,...,...
2886,3472,Tomato - Tricolor Cherry,2
2887,3472,Tomatoes Tear Drop,1
2888,3472,Veal - Slab Bacon,1
2889,3472,"Water - Mineral, Natural",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [210]:
ranked = records.groupby(["CustomerID","ProductName"])["Quantity"].sum().reset_index().sort_values(by=["Quantity"],ascending=False)


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [227]:
merged = pd.merge(ranked,pivot,on="ProductName")
filtered = merged[merged[CUSTOMERID]==0].head(5)
for i in filtered["ProductName"]:
    print(i)

Pork - Kidney
Cheese - Boursin, Garlic / Herbs
Juice - Lime
Veal - Sweetbread
Mushrooms - Black, Dried


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [242]:
recommendations = {}
customers = pivot.columns.unique()

for c in customers: 
    similarities = distances[c].sort_values(ascending=False)[1:]
    top_5 = similarities[:5]
    records = grouped[grouped["CustomerID"].isin(top_5.index.to_list())]
    ranked = records.groupby(["CustomerID","ProductName"])["Quantity"].sum().reset_index().sort_values(by=["Quantity"],ascending=False)
    merged = pd.merge(ranked,pivot,on="ProductName")
    filtered = merged[merged[CUSTOMERID]==0].head(5)
    recommendations[c]=list(filtered["ProductName"])


In [243]:
recommendations[33]

['Salsify, Organic',
 'Salsify, Organic',
 'Butter - Unsalted',
 'Tofu - Firm',
 'Mushrooms - Black, Dried']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [244]:
pd.DataFrame.from_dict(recommendations,orient="index")

,0,1,2,3,4
33,"Salsify, Organic","Salsify, Organic",Butter - Unsalted,Tofu - Firm,"Mushrooms - Black, Dried"
200,Pork - Kidney,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,"Pepper - Black, Whole",Vanilla Beans
264,Pork - Kidney,Cheese - Parmesan Grated,Cheese - Parmesan Grated,Lime Cordial - Roses,Sugar - Fine
356,Table Cloth 81x81 White,Soup - Campbells Tomato Ravioli,Veal - Sweetbread,Veal - Sweetbread,Beets - Mini Golden
412,"Pepper - Black, Whole","Pepper - Black, Whole",Wine - Vidal Icewine Magnotta,Soup - Campbells Tomato Ravioli,"Pepper - White, Ground"
...,...,...,...,...,...
97928,Wine - Vidal Icewine Magnotta,Beans - Kidney White,Beans - Kidney White,Beans - Kidney White,Bay Leaf
98069,Soupfoamcont12oz 112con,Yoghurt Tubes,Yoghurt Tubes,Milk - 1%,Milk - 1%
98159,Onion Powder,Eggplant - Asian,Eggplant - Asian,"Soup - Campbells, Lentil","Wine - Red, Harrow Estates, Cab"
98185,Crackers - Trio,Beef - Montreal Smoked Brisket,Cheese - Taleggio D.o.p.,Cheese - Taleggio D.o.p.,Whmis - Spray Bottle Trigger


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [245]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'jaccard'))), 
                         index=pivot.columns, columns=pivot.columns)
distances

recommendations = {}
customers = pivot.columns.unique()

for c in customers: 
    similarities = distances[c].sort_values(ascending=False)[1:]
    top_5 = similarities[:5]
    records = grouped[grouped["CustomerID"].isin(top_5.index.to_list())]
    ranked = records.groupby(["CustomerID","ProductName"])["Quantity"].sum().reset_index().sort_values(by=["Quantity"],ascending=False)
    merged = pd.merge(ranked,pivot,on="ProductName")
    filtered = merged[merged[CUSTOMERID]==0].head(5)
    recommendations[c]=list(filtered["ProductName"])

In [246]:
pd.DataFrame.from_dict(recommendations,orient="index")

,0,1,2,3,4
33,"Salsify, Organic","Salsify, Organic",Veal - Sweetbread,Veal - Sweetbread,Sunflower Seed Raw
200,Pork - Kidney,Ice Cream Bar - Oreo Cone,Vol Au Vents,"Cheese - Boursin, Garlic / Herbs",Papayas
264,Cheese - Parmesan Grated,"Mushrooms - Black, Dried",Lettuce - Spring Mix,Lettuce - Spring Mix,Potatoes - Idaho 100 Count
356,Sausage - Breakfast,Sausage - Breakfast,Duck - Breast,Skirt - 29 Foot,Tea - Earl Grey
412,Smirnoff Green Apple Twist,Watercress,Cumin - Whole,Cumin - Whole,"Pepper - Black, Whole"
...,...,...,...,...,...
97928,Scampi Tail,"Crab - Dungeness, Whole",Vinegar - Sherry,"Wine - White, Schroder And Schyl","Wine - White, Schroder And Schyl"
98069,Scampi Tail,Scampi Tail,Fondant - Icing,Ecolab - Lime - A - Way 4/4 L,Campari
98159,Ecolab - Lime - A - Way 4/4 L,Ecolab - Lime - A - Way 4/4 L,Bread - Italian Corn Meal Poly,Chicken - Soup Base,Tea - Herbal Sweet Dreams
98185,Coffee - Irish Cream,Coffee - Irish Cream,Tomato - Tricolor Cherry,Chicken - Wieners,Ecolab - Lime - A - Way 4/4 L
